# Package for WRC Results Scrape

This notebook contains most of the ingredients for a package that can be used to scrape results data from current and archived WRC Live Timing pages.

At the moment, the script will only happily downloaded the complete set of results for a particular rally. The code needs updating to allow:

- loading data for a completed stage into a database that already contains results for other completed stages;
- incremental loads of data to upsert data into the database from an ongoing stage;

In [ ]:
#https://www.wrc.com/service/sasCacheApi.php?route=rallies%2F40%2Fitinerary

In [17]:
url_base='http://www.wrc.com/service/sasCacheApi.php?route={stub}'

In [18]:
#Call a resource by ID
wrcapi='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byId?id=%22{}%22' #requires resource ID

In [19]:
stubs = { 'itinerary': 'rallies/{rallyId}/itinerary',
          'startlists': 'rallies/{rallyId}/entries',
         'penalties': 'rallies/{rallyId}/penalties',
         'retirements': 'rallies/{rallyId}/retirements',
         'stagewinners':'rallies/{rallyId}/stagewinners',
         'overall':'stages/{stageId}/results?rallyId={rallyId}',
         'split_times':'stages/{stageId}/splittimes?rallyId={rallyId}',
         'stage_times_stage':'stages/{stageId}/stagetimes?rallyId={rallyId}',
         'stage_times_overall':'stages/{stageId}/results?rallyId={rallyId}',
         'championship':'championships/{championshipId}',
         'championship_results':'championships/{championshipId}/results' }

In [20]:
import requests
import re
import json
from bs4 import BeautifulSoup 

import sqlite3

import pandas as pd
from pandas.io.json import json_normalize

## Database Schema

The following tables are literal mappings from flattened JSON datafiles published by the WRC.

The data model (primary and foreign key relationships) is derived by observation.

In [21]:
#SQL in wrcResults.sql
setup_q='''
CREATE TABLE "itinerary_event" (
  "eventId" INTEGER,
  "itineraryId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "priority" INTEGER
);
CREATE TABLE "itinerary_legs" (
  "itineraryId" INTEGER,
  "itineraryLegId" INTEGER PRIMARY KEY,
  "legDate" TEXT,
  "name" TEXT,
  "order" INTEGER,
  "startListId" INTEGER,
  "status" TEXT,
  FOREIGN KEY ("itineraryId") REFERENCES "itinerary_event" ("itineraryId")
);
CREATE TABLE "itinerary_sections" (
  "itineraryLegId" INTEGER,
  "itinerarySectionId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "order" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_stages" (
  "code" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "name" TEXT,
  "number" INTEGER,
  "stageId" INTEGER PRIMARY KEY,
  "stageType" TEXT,
  "status" TEXT,
  "timingPrecision" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_controls" (
  "code" TEXT,
  "controlId" INTEGER PRIMARY KEY,
  "controlPenalties" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "firstCarDueDateTime" TEXT,
  "firstCarDueDateTimeLocal" TEXT,
  "location" TEXT,
  "stageId" INTEGER,
  "status" TEXT,
  "targetDuration" TEXT,
  "targetDurationMs" INTEGER,
  "timingPrecision" TEXT,
  "type" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  "roundingPolicy" TEXT,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "startlists" (
  "codriver.abbvName" TEXT,
  "codriver.code" TEXT,
  "codriver.country.countryId" INTEGER,
  "codriver.country.iso2" TEXT,
  "codriver.country.iso3" TEXT,
  "codriver.country.name" TEXT,
  "codriver.countryId" INTEGER,
  "codriver.firstName" TEXT,
  "codriver.fullName" TEXT,
  "codriver.lastName" TEXT,
  "codriver.personId" INTEGER,
  "codriverId" INTEGER,
  "driver.abbvName" TEXT,
  "driver.code" TEXT,
  "driver.country.countryId" INTEGER,
  "driver.country.iso2" TEXT,
  "driver.country.iso3" TEXT,
  "driver.country.name" TEXT,
  "driver.countryId" INTEGER,
  "driver.firstName" TEXT,
  "driver.fullName" TEXT,
  "driver.lastName" TEXT,
  "driver.personId" INTEGER,
  "driverId" INTEGER,
  "eligibility" TEXT,
  "entrant.entrantId" INTEGER,
  "entrant.logoFilename" TEXT,
  "entrant.name" TEXT,
  "entrantId" INTEGER,
  "entryId" INTEGER PRIMARY KEY,
  "eventId" INTEGER,
  "group.name" TEXT,
  "groupId" INTEGER,
  "group.groupId" INTEGER,
  "identifier" TEXT,
  "manufacturer.logoFilename" TEXT,
  "manufacturer.manufacturerId" INTEGER,
  "manufacturer.name" TEXT,
  "manufacturerId" INTEGER,
  "priority" TEXT,
  "status" TEXT,
  "tag" TEXT,
  "tag.name" TEXT,
  "tag.tagId" INTEGER,
  "tagId" INTEGER,
  "tyreManufacturer" TEXT,
  "vehicleModel" TEXT,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "roster" (
  "fiasn" INTEGER,
  "code" TEXT,
  "sas-entryid" INTEGER PRIMARY KEY,
  "roster_num" INTEGER,
  FOREIGN KEY ("sas-entryid") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "startlist_classes" (
  "eventClassId" INTEGER,
  "eventId" INTEGER,
  "name" TEXT,
  "entryId" INTEGER,
  PRIMARY KEY ("eventClassId","entryId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "penalties" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "penaltyDuration" TEXT,
  "penaltyDurationMs" INTEGER,
  "penaltyId" INTEGER PRIMARY KEY,
  "reason" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "retirements" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "reason" TEXT,
  "retirementDateTime" TEXT,
  "retirementDateTimeLocal" TEXT,
  "retirementId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stagewinners" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "stageId" INTEGER,
  "stageName" TEXT,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId")
);
CREATE TABLE "stage_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "split_times" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "splitDateTime" TEXT,
  "splitDateTimeLocal" TEXT,
  "splitPointId" INTEGER,
  "splitPointTimeId" INTEGER PRIMARY KEY,
  "stageTimeDuration" TEXT,
  "stageTimeDurationMs" REAL,
  "startDateTime" TEXT,
  "startDateTimeLocal" TEXT,
  "stageId" INTEGER,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_stage" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "position" INTEGER,
  "source" TEXT,
  "stageId" INTEGER,
  "stageTimeId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "championship_lookup" (
  "championshipId" INTEGER PRIMARY KEY,
  "fieldFiveDescription" TEXT,
  "fieldFourDescription" TEXT,
  "fieldOneDescription" TEXT,
  "fieldThreeDescription" TEXT,
  "fieldTwoDescription" TEXT,
  "name" TEXT,
  "seasonId" INTEGER,
  "type" TEXT,
  "_codeClass" TEXT,
  "_codeTyp" TEXT
);
CREATE TABLE "championship_results" (
  "championshipEntryId" INTEGER,
  "championshipId" INTEGER,
  "dropped" INTEGER,
  "eventId" INTEGER,
  "pointsBreakdown" TEXT,
  "position" INTEGER,
  "publishedStatus" TEXT,
  "status" TEXT,
  "totalPoints" INTEGER,
  PRIMARY KEY ("championshipEntryId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_codrivers" (
  "championshipEntryId" INTEGER PRIMARY KEY,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_entries_manufacturers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" INTEGER,
  "Name" TEXT,
  "LogoFileName" TEXT,
  "Manufacturer" TEXT,
  "manufacturerId" INTEGER,
  "personId" TEXT,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_rounds" (
  "championshipId" INTEGER,
  "eventId" INTEGER,
  "order" INTEGER,
  PRIMARY KEY ("championshipId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_events" (
  "categories" TEXT,
  "clerkOfTheCourse" TEXT,
  "country.countryId" INTEGER,
  "country.iso2" TEXT,
  "country.iso3" TEXT,
  "country.name" TEXT,
  "countryId" INTEGER,
  "eventId" INTEGER PRIMARY KEY,
  "finishDate" TEXT,
  "location" TEXT,
  "mode" TEXT,
  "name" TEXT,
  "organiserUrl" TEXT,
  "slug" TEXT,
  "startDate" TEXT,
  "stewards" TEXT,
  "surfaces" TEXT,
  "templateFilename" TEXT,
  "timeZoneId" TEXT,
  "timeZoneName" TEXT,
  "timeZoneOffset" INTEGER,
  "trackingEventId" INTEGER ,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_drivers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "event_metadata" (
  "_id" TEXT,
  "availability" TEXT,
  "date-finish" TEXT,
  "date-start" TEXT,
  "gallery" TEXT,
  "hasdata" TEXT,
  "hasfootage" TEXT,
  "hasvideos" TEXT,
  "id" TEXT,
  "info-based" TEXT,
  "info-categories" TEXT,
  "info-date" TEXT,
  "info-flag" TEXT,
  "info-surface" TEXT,
  "info-website" TEXT,
  "kmlfile" TEXT,
  "logo" TEXT,
  "name" TEXT,
  "org-website" TEXT,
  "poi-Klo im Wald" TEXT,
  "poilistid" TEXT,
  "position" TEXT,
  "rosterid" TEXT,
  "sas-eventid" TEXT,
  "sas-itineraryid" TEXT,
  "sas-rallyid" TEXT,
  "sas-trackingid" TEXT,
  "sitid" TEXT,
  "testid" TEXT,
  "thumbnail" TEXT,
  "time-zone" TEXT,
  "tzoffset" TEXT,
  "year" INTEGER
);


'''


#conn = sqlite3.connect('wrc18_test1keys.db')
#c = conn.cursor()
#c.executescript(setup_q)

In [22]:
setup_views_q = '''
'''

In [7]:
#meta={'rallyId':None, 'stages':[], 'championshipId':None }

In [23]:
def _getEventMetadata():
    ''' Get event metadata as JSON data feed from WRC API. '''
    url='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Event%22&maxdepth=1'
    eventmeta = requests.get(url).json()
    return eventmeta

def getEventMetadata():
    ''' Get a list of events from WRC as a flat pandas dataframe.
        Itinerary / event data is only available for rallies starting in 2018. '''
    eventMetadata = json_normalize(_getEventMetadata(),
                                   record_path='_meta',
                                   meta='_id'  ).drop_duplicates().pivot('_id', 'n','v').reset_index()

    eventMetadata['date-finish']=pd.to_datetime(eventMetadata['date-finish'])
    eventMetadata['date-start']=pd.to_datetime(eventMetadata['date-start'])
    eventMetadata['year'] = eventMetadata['date-start'].dt.year
    return eventMetadata


getEventMetadata()

n,_id,availability,date-finish,date-start,gallery,hasdata,hasvideos,id,info-based,info-categories,...,sas-eventid,sas-itineraryid,sas-rallyid,sas-trackingid,sitid,testid,thumbnail,time-zone,tzoffset,year
0,029e41e3-afff-44ad-950f-9af3dccf06be,now,2015-08-02,2015-07-30,NaN,NaN,true,neste-oil-rally-finland,Jyväskylä,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,143,NaN,NaN,Helsinki,NaN,2015
1,03d002d1-82d8-499c-8252-bdc36685ffd3,now,2015-10-25,2015-10-22,NaN,NaN,true,rallyracc-rally-de-espana,Salou,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,151,NaN,NaN,Madrid,NaN,2015
2,0919ef82-a65f-4115-8372-1364be04a219,now,2015-10-04,2015-10-02,NaN,NaN,true,rallye-de-france-alsace,Corte,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,149,NaN,NaN,Paris,NaN,2015
3,0b24651f-0eeb-4e44-81ed-2a5aa3a3eba9,now,2017-04-30,2017-04-27,NaN,NaN,true,rally-argentina,Villa Carlos Paz,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,224,NaN,NaN,Buenos Aires,-14400000,2017
4,0d6cf890-381e-4010-9fd8-36d942ac80d2,now,2017-02-12,2017-02-09,NaN,NaN,true,rally-sweden,Karlstad,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,218,NaN,NaN,Stockholm,NaN,2017
5,0f3fa7ea-f3f1-493a-87d5-5cb3f0fad39e,now,2018-09-16,2018-09-13,NaN,NaN,true,NaN,Marmaris,NaN,...,35,114,39,2596,35,NaN,NaN,NaN,10800000,2018
6,117b1be7-1d26-447f-a405-878da1112436,now,2014-08-24,2014-08-21,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,91,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Europe/Berlin,NaN,2014
7,182ce929-8bda-4acf-9fd1-c178d9d68482,now,2018-06-10,2018-06-07,NaN,NaN,true,NaN,Sardegna,NaN,...,32,83,36,2593,32,NaN,NaN,NaN,3600000,2018
8,1a3cdf87-4d91-435c-a1f9-7aab556fc3b1,now,2017-08-20,2017-08-17,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,234,NaN,NaN,Berlin,3600000,2017
9,1d7c1500-42fd-4045-b447-1cc8006245b6,NaN,2014-06-08,2014-06-05,NaN,NaN,NaN,rally-italia-sardegna,Alghero,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,82,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Rome,NaN,2014


In [95]:
def _getRallyIDs2(year=2018):
    em=getEventMetadata()
    em = em[em['year']==year][['name','sas-rallyid', 'kmlfile', 'date-start']].reset_index(drop=True).dropna()
    em['stub']=em['kmlfile'].apply(lambda x: x.split('_')[0])
    return em

def getRallyIDs2(year=2018):
    em = _getRallyIDs2(year=2018)
    return em[['stub','sas-rallyid']].set_index('stub').to_dict()['sas-rallyid']

def listRallies2(year=2018):
    return getRallyIDs2(year)

In [96]:
listRallies2()

{'turkey': '39',
 'sardegna': '36',
 'france': '33',
 'spain': '41',
 'finland': '37',
 'portugal': '35',
 'germany': '38',
 'mexico': '32',
 'argentina': '34',
 'australia': '42',
 'montecarlo': '30',
 'uk': '40',
 'sweden': '31'}

In [8]:
listRallies2()

{'turkey': '39',
 'sardegna': '36',
 'france': '33',
 'spain': '41',
 'finland': '37',
 'portugal': '35',
 'germany': '38',
 'mexico': '32',
 'argentina': '34',
 'montecarlo': '30',
 'uk': '40',
 'sweden': '31'}

In [16]:
#rallyIDs = getRallyIDs2()
#rallyIDs

In [25]:
def nvToDict(nvdict, key='n',val='v', retdict=None):
    if retdict is None:
        retdict={nvdict[key]:nvdict[val]}
    else:
        retdict[nvdict[key]]=nvdict[val]
    return retdict
#assert nvToDict({'n': "id",'v': "adac-rallye-deutschland"}) == {'id': 'adac-rallye-deutschland'}

In [18]:
 #getEventMetadata()['rosterid'].iloc[0]

In [26]:
'''
def set_rallyId(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta
'''

def set_rallyId2(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs2()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta

In [109]:
getEventMetadata()

n,_id,availability,date-finish,date-start,gallery,hasdata,hasvideos,id,info-based,info-categories,...,sas-eventid,sas-itineraryid,sas-rallyid,sas-trackingid,sitid,testid,thumbnail,time-zone,tzoffset,year
0,029e41e3-afff-44ad-950f-9af3dccf06be,now,2015-08-02,2015-07-30,NaN,NaN,true,neste-oil-rally-finland,Jyväskylä,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,143,NaN,NaN,Helsinki,NaN,2015
1,03d002d1-82d8-499c-8252-bdc36685ffd3,now,2015-10-25,2015-10-22,NaN,NaN,true,rallyracc-rally-de-espana,Salou,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,151,NaN,NaN,Madrid,NaN,2015
2,0919ef82-a65f-4115-8372-1364be04a219,now,2015-10-04,2015-10-02,NaN,NaN,true,rallye-de-france-alsace,Corte,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,149,NaN,NaN,Paris,NaN,2015
3,0b24651f-0eeb-4e44-81ed-2a5aa3a3eba9,now,2017-04-30,2017-04-27,NaN,NaN,true,rally-argentina,Villa Carlos Paz,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,224,NaN,NaN,Buenos Aires,-14400000,2017
4,0d6cf890-381e-4010-9fd8-36d942ac80d2,now,2017-02-12,2017-02-09,NaN,NaN,true,rally-sweden,Karlstad,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,218,NaN,NaN,Stockholm,NaN,2017
5,0f3fa7ea-f3f1-493a-87d5-5cb3f0fad39e,now,2018-09-16,2018-09-13,NaN,NaN,true,NaN,Marmaris,NaN,...,35,114,39,2596,35,NaN,NaN,NaN,10800000,2018
6,117b1be7-1d26-447f-a405-878da1112436,now,2014-08-24,2014-08-21,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,91,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Europe/Berlin,NaN,2014
7,182ce929-8bda-4acf-9fd1-c178d9d68482,now,2018-06-10,2018-06-07,NaN,NaN,true,NaN,Sardegna,NaN,...,32,83,36,2593,32,NaN,NaN,NaN,3600000,2018
8,1a3cdf87-4d91-435c-a1f9-7aab556fc3b1,now,2017-08-20,2017-08-17,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,234,NaN,NaN,Berlin,3600000,2017
9,1d7c1500-42fd-4045-b447-1cc8006245b6,NaN,2014-06-08,2014-06-05,NaN,NaN,NaN,rally-italia-sardegna,Alghero,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,82,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Rome,NaN,2014


In [27]:
roster_id='bab64d15-4691-4561-a6bf-7284f3bd85f9'
import requests
#roster_json = requests.get( '{}&maxdepth=2'.format(wrcapi.format(roster_id),) ).json()
#roster_json   

#TO CHECK - is the sas-entryid the entryid we use elsewhere?

#This comes from event metadata
def _getRoster(roster_id):
    roster_json = requests.get(wrcapi.format(roster_id) ).json()
    roster=json_normalize(roster_json)
    
    aa=json_normalize(roster_json, record_path='_dchildren')
    zz=json_normalize(roster_json['_dchildren'],record_path=['_meta'], meta='_id').pivot('_id', 'n','v').reset_index()
    zz=pd.merge(zz,aa[['_id','name','type']], on='_id')[['fiasn','filename','sas-entryid','name']]
    zz.columns = ['fiasn','code','sas-entryid','roster_num']
    #defensive?
    zz = zz.dropna(subset=['sas-entryid'])
    return zz

def getRoster(meta):
    em = getEventMetadata()
    roster_id= em[em['sas-rallyid']==meta['rallyId']]['rosterid'].iloc[0]
    return _getRoster(roster_id)

In [28]:
def getItinerary(meta):
    ''' Get event itinerary. Also updates the stages metadata. '''
    itinerary_json=requests.get( url_base.format(stub=stubs['itinerary'].format(**meta) ) ).json()
    itinerary_event = json_normalize(itinerary_json).drop('itineraryLegs', axis=1)
    
    #meta='eventId' for eventId
    itinerary_legs = json_normalize(itinerary_json, 
                                    record_path='itineraryLegs').drop('itinerarySections', axis=1)
    #meta='eventId' for eventId
    itinerary_sections = json_normalize(itinerary_json,
                                        ['itineraryLegs', 'itinerarySections']).drop(['stages','controls'],axis=1)

    itinerary_stages=json_normalize(itinerary_json['itineraryLegs'],
                                    ['itinerarySections','stages'],
                                   meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    meta['stages']=itinerary_stages['stageId'].tolist()
    #Should do this a pandas idiomatic way
    #meta['_stages']=zip(itinerary_stages['stageId'].tolist(),
     #                   itinerary_stages['code'].tolist(),
     #                   itinerary_stages['status'].tolist())
    meta['_stages'] = itinerary_stages[['stageId','code','status']].set_index('code').to_dict(orient='index')
    itinerary_controls=json_normalize(itinerary_json['itineraryLegs'], 
                                  ['itinerarySections','controls'] ,
                                     meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    itinerary_controls['stageId'] = itinerary_controls['stageId'].fillna(-1).astype(int)
    
    return itinerary_event, itinerary_legs, itinerary_sections, itinerary_stages, itinerary_controls

In [23]:
#a,b,c,d,e = getItinerary(meta)

In [29]:
def _get_single_json_table(meta, stub):
    _json = requests.get( url_base.format(stub=stubs[stub].format(**meta) ) ).json()
    return json_normalize(_json)

In [25]:
#meta =  set_rallyId(name, year)

#startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
#ff=[]
#for f in startlists_json:
#    if f['manufacturer']['logoFilename'] is None:
#        f['manufacturer']['logoFilename']=''
#    if f['entrant']['logoFilename'] is None:
#        f['entrant']['logoFilename']='' 
#    ff.append(f)
#ff

In [26]:
#startlists = json_normalize(ff).drop('eventClasses', axis=1)

In [30]:
def get_startlists(meta):
    startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
    ff=[]
    for f in startlists_json:
        if f['manufacturer']['logoFilename'] is None:
            f['manufacturer']['logoFilename']=''
        if f['entrant']['logoFilename'] is None:
            f['entrant']['logoFilename']='' 
        ff.append(f)
    startlists = json_normalize(ff).drop('eventClasses', axis=1)
    startlist_classes = json_normalize(ff,['eventClasses'], 'entryId' )
    #startlists = json_normalize(startlists_json).drop('eventClasses', axis=1)
    #startlist_classes = json_normalize(startlists_json,['eventClasses'], 'entryId' )
    
    return startlists, startlist_classes 

In [31]:
def get_penalties(meta):
    ''' Get the list of penalties for a specified event. '''
    penalties = _get_single_json_table(meta, 'penalties')
    return penalties

In [32]:
def get_retirements(meta):
    ''' Get the list of retirements for a specified event. '''
    retirements = _get_single_json_table(meta, 'retirements')
    return retirements

In [33]:
def get_stagewinners(meta):
    ''' Get the stage winners table for a specified event. '''
    stagewinners = _get_single_json_table(meta, 'stagewinners')
    return stagewinners

In [34]:
def _single_stage(meta2, stub, stageId):
    ''' For a single stageId, get the requested resource. '''
    meta2['stageId']=stageId
    _json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
    _df = json_normalize(_json)
    _df['stageId'] = stageId
    return _df

def _stage_iterator(meta, stub, stage=None):
    ''' Iterate through a list of stageId values and get requested resource. '''
    meta2={'rallyId':meta['rallyId']}
    df = pd.DataFrame()
    #If stage is None get data for all stages
    if stage is not None:
        stages=[]
        #If we have a single stage (specified in form SS4) get it
        if isinstance(stage,str) and stage in meta['_stages']:
            stages.append(meta['_stages'][stage]['stageId'])
        #If we have a list of stages (in form ['SS4','SS5']) get them all
        elif isinstance(stage, list):
            for _stage in stage:
                if isinstance(_stage,str) and _stage in meta['_stages']:
                    stages.append(meta['_stages'][_stage]['stageId'])
                elif _stage in meta['stages']:
                    stages.append(_stage)
    else:
        stages = meta['stages']
        
    #Get data for required stages
    for stageId in stages:
        #meta2['stageId']=stageId
        #_json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
        #_df = json_normalize(_json)
        #_df['stageId'] = stageId
        _df = _single_stage(meta2, stub, stageId)
        df = pd.concat([df, _df], sort=False)
    return df.reset_index(drop=True)

In [35]:
def get_overall(meta, stage=None):
    ''' Get the overall results table for all stages on an event or a specified stage. '''
    stage_overall = _stage_iterator(meta, 'overall', stage)
    return stage_overall

In [36]:
def get_splitTimes(meta, stage=None):
    ''' Get split times table for all stages on an event or a specified stage. '''
    split_times = _stage_iterator(meta, 'split_times', stage)
    return split_times

In [37]:
def get_stage_times_stage(meta, stage=None):
    ''' Get stage times table for all stages on an event or a specified stage. '''
    stage_times_stage = _stage_iterator(meta, 'stage_times_stage', stage)
    return stage_times_stage

In [38]:
def get_stage_times_overall(meta,stage=None):
    ''' Get overall stage times table for all stages on an event or a specified stage. '''
    stage_times_overall = _stage_iterator(meta, 'stage_times_overall', stage)
    return stage_times_overall

In [39]:
#There must be a JSON/API way of getting this rather than having to fish for it
def _get_championship_codes(url=None):
    if url is None:
        url = 'http://www.wrc.com/en/wrc/results/championship-standings/page/4176----.html'
    html2=requests.get(url).text
    m = re.search("var championshipClasses = (.*?);", html2, re.DOTALL)
    mm=m.group(1).replace('\n','').replace("'",'"')
    d=json.loads(mm)
    #https://stackoverflow.com/a/35758583/454773
    championshipClasses={k.replace(' ', ''): v for k, v in d.items()}
    return championshipClasses

In [40]:
#_get_championship_codes()

In [41]:
def championship_tables(champ_class=None, champ_typ=None):
    ''' Get all championship tables in a particular championship and / or class. '''
    #if championship is None then get all
    championship_lookup = pd.DataFrame()
    championship_entries_all = {}
    championship_rounds = pd.DataFrame()
    championship_events = pd.DataFrame()
    championship_results = pd.DataFrame()
    
    championship_codes = _get_championship_codes()
    _class_codes = championship_codes.keys() if champ_class is None else [champ_class]
    for champClass in _class_codes:
        _champ_typ = championship_codes[champClass].keys() if champ_typ is None else [champ_typ]
        for champType in _champ_typ:
            if champType not in championship_entries_all:
                championship_entries_all[champType] = pd.DataFrame()
            
            champ_num = championship_codes[champClass][champType]
            meta2={'championshipId': champ_num}
            
            championship_json=requests.get( url_base.format(stub=stubs['championship'].format(**meta2) ) ).json()

            _championship_lookup = json_normalize(championship_json).drop(['championshipEntries','championshipRounds'], axis=1)
            _championship_lookup['_codeClass'] = champClass
            _championship_lookup['_codeTyp'] = champType
            championship_lookup = pd.concat([championship_lookup,_championship_lookup],sort=True)
            
            championships={}
            championship_dict = _championship_lookup.to_dict()
            championships[champ_num] = {c:championship_dict[c][0] for c in championship_dict}
            renamer={c.replace('Description',''):championships[champ_num][c] for c in championships[champ_num] if c.startswith('field')}            
            _championship_entries = json_normalize(championship_json,['championshipEntries'] )
            _championship_entries = _championship_entries.rename(columns=renamer)
            _championship_entries = _championship_entries[[c for c in _championship_entries.columns if c!='']]
            #pd.concat sort=False to retain current behaviour
            championship_entries_all[champType] = pd.concat([championship_entries_all[champType],_championship_entries],sort=False)
            
            _championship_rounds = json_normalize(championship_json,['championshipRounds'] ).drop('event', axis=1)
            championship_rounds = pd.concat([championship_rounds,_championship_rounds],sort=False).drop_duplicates()
            
            _events_json = json_normalize(championship_json,['championshipRounds' ])['event']
            _championship_events = json_normalize(_events_json)
            championship_events = pd.concat([championship_events,_championship_events],sort=False).drop_duplicates()
        
            _championship_results = _get_single_json_table(meta2, 'championship_results')
            championship_results = pd.concat([championship_results, _championship_results],sort=False)
    
    for k in championship_entries_all:
        championship_entries_all[k].reset_index(drop=True)
        if k in ['Driver', 'Co-Driver']:
            championship_entries_all[k] = championship_entries_all[k].rename(columns={'TyreManufacturer':'ManufacturerTyre'})
    
    return championship_lookup.reset_index(drop=True), \
            championship_results.reset_index(drop=True), \
            championship_entries_all, \
            championship_rounds.reset_index(drop=True), \
            championship_events.reset_index(drop=True)


In [39]:
#a,b,c,d,e = championship_tables()

## Usage

In [40]:
#listRallies2()

In [42]:
def cleardbtable(conn, table):
    ''' Clear the table whilst retaining the table definition '''
    c = conn.cursor()
    c.execute('DELETE FROM "{}"'.format(table))
    
def dbfy(conn, df, table, if_exists='append', index=False, clear=False, **kwargs):
    ''' Save a dataframe as a SQLite table.
        Clearing or replacing a table will first empty the table of entries but retain the structure. '''
    if if_exists=='replace':
        clear=True
        if_exists='append'
    if clear: cleardbtable(conn, table)
    df.to_sql(table,conn,if_exists=if_exists,index=index)

In [43]:
def save_rally(meta, conn, stage=None):
    ''' Save all tables associated with a particular rally. '''
    
    if stage is None:
        
        roster = getRoster(meta)
        dbfy(conn, roster, 'roster')

        itinerary_event, itinerary_legs, itinerary_sections, \
        itinerary_stages, itinerary_controls = getItinerary(meta)

        dbfy(conn, itinerary_event, 'itinerary_event')
        dbfy(conn, itinerary_legs, 'itinerary_legs')
        dbfy(conn, itinerary_sections, 'itinerary_sections')
        dbfy(conn, itinerary_stages, 'itinerary_stages')
        dbfy(conn, itinerary_controls, 'itinerary_controls')

        startlists, startlist_classes = get_startlists(meta)
        dbfy(conn, startlists, 'startlists')
        dbfy(conn, startlist_classes, 'startlist_classes')

        #These need to be upserted
        penalties = get_penalties(meta)
        dbfy(conn, penalties, 'penalties')

        retirements = get_retirements(meta)
        dbfy(conn, retirements, 'retirements')

        stagewinners = get_stagewinners(meta)
        dbfy(conn, stagewinners, 'stagewinners')


    stage_overall = get_overall(meta, stage)
    dbfy(conn, stage_overall, 'stage_overall')

    split_times = get_splitTimes(meta, stage)
    dbfy(conn, split_times, 'split_times')
    
    stage_times_stage = get_stage_times_stage(meta, stage)
    dbfy(conn, stage_times_stage, 'stage_times_stage')
    
    stage_times_overall = get_stage_times_overall(meta, stage)
    dbfy(conn, stage_times_overall, 'stage_times_overall')

    
def save_championship(meta, conn):
    ''' Save all championship tables for a particular year. '''
    championship_lookup, championship_results, _championship_entries_all, \
        championship_rounds, championship_events = championship_tables()
        
    championship_entries_drivers = _championship_entries_all['Driver']
    championship_entries_codrivers = _championship_entries_all['Co-Driver']
    championship_entries_manufacturers = _championship_entries_all['Manufacturers']
    
    dbfy(conn, championship_lookup, 'championship_lookup', clear=True)
    dbfy(conn, championship_results, 'championship_results', clear=True)
    dbfy(conn, championship_entries_drivers, 'championship_entries_drivers', clear=True)
    dbfy(conn, championship_entries_codrivers, 'championship_entries_codrivers', clear=True)
    dbfy(conn, championship_entries_manufacturers, 'championship_entries_manufacturers', clear=True)
    dbfy(conn, championship_rounds, 'championship_rounds', clear=True)
    dbfy(conn, championship_events, 'championship_events', clear=True)

def get_one(rally, stage, dbname='wrc18_test1.db', year=2018):
    conn = sqlite3.connect(dbname)
    meta =  set_rallyId2(rally, year)
    getItinerary(meta) #to update meta
    print(meta)
    save_rally(meta, conn, stage)
    
def get_all(rally, dbname='wrc18_test1.db', year=2018):
    
    conn = sqlite3.connect(dbname)
    
    meta =  set_rallyId2(rally, year)
    
    save_rally(meta, conn)
    save_championship(meta, conn)


In [44]:
#listRallies()
listRallies2()

{'turkey': '39',
 'sardegna': '36',
 'france': '33',
 'spain': '41',
 'finland': '37',
 'portugal': '35',
 'germany': '38',
 'mexico': '32',
 'argentina': '34',
 'australia': '42',
 'montecarlo': '30',
 'uk': '40',
 'sweden': '31'}

In [45]:
#dbname='wrc18.db'
year = 2018
name = 'australia'
dbname='australia18.db'

In [46]:
import os

#For some reason, these don't seem to get set / picked up correctly from a notebook?
os.environ["WRC_RESULTS_NAME"] = name
os.environ["WRC_RESULTS_DBNAME"] = dbname
os.environ["WRC_RESULTS_YEAR"] = str(year)

In [114]:
#set_rallyId2('uk', 2018)

{'rallyId': '40', 'stages': [], 'championshipId': None, 'rally_name': 'uk'}

In [68]:
#rr=get_retirements(meta)
#rr.head()

In [70]:
# TO DO - ability to top up just the stage we need

In [15]:
#set_rallyId("Finland",2018)
#set_rallyId2("australia",2018)

NameError: name 'set_rallyId2' is not defined

In [50]:
getEventMetadata().columns

Index(['_id', 'availability', 'date-finish', 'date-start', 'gallery',
       'hasdata', 'hasfootage', 'hasvideos', 'id', 'info-based',
       'info-categories', 'info-date', 'info-flag', 'info-surface',
       'info-website', 'kmlfile', 'logo', 'name', 'org-website',
       'poi-Klo im Wald', 'poilistid', 'position', 'rosterid', 'sas-eventid',
       'sas-itineraryid', 'sas-rallyid', 'sas-trackingid', 'sitid', 'testid',
       'thumbnail', 'time-zone', 'tzoffset', 'year'],
      dtype='object', name='n')

In [92]:
#full run
#dbname = 'wrc18.db'

In [98]:
#new db
!mv $dbname old-$dbname
!rm $dbname
conn = sqlite3.connect(dbname)
c = conn.cursor()
c.executescript(setup_q)
c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"

dbfy(conn, getEventMetadata(), 'event_metadata')

pd.read_sql(q,conn)

rm: wrc18.db: No such file or directory


/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [52]:
#DEBUG
#for example: OperationalError: table event_metadata has no column named hasfootage
#

In [53]:
#itinerary_event

In [54]:
#itinerary_event

In [91]:
#set_rallyId( name,year )
get_all(name, dbname=dbname, year=year )

In [99]:
#full run:
for name in listRallies2():
    print('trying {}'.format(name))
    get_all(name, dbname=dbname, year=year )

trying turkey
trying sardegna
trying france
trying spain
trying finland
trying portugal
trying germany
trying mexico
trying argentina
trying australia
trying montecarlo
trying uk
trying sweden


In [70]:
#PK issues?
#upsert issues? Pandas doesn't support upsert
get_one(name, 'SS12', dbname=dbname, year=year)

{'rallyId': '41', 'stages': [823, 828, 830, 831, 835, 819, 833, 836, 820, 821, 824, 826, 827, 834, 832, 829, 825, 822], 'championshipId': None, 'rally_name': 'Spain', '_stages': {'SS1': {'stageId': 823, 'status': 'Completed'}, 'SS2': {'stageId': 828, 'status': 'Completed'}, 'SS3': {'stageId': 830, 'status': 'Completed'}, 'SS4': {'stageId': 831, 'status': 'Interrupted'}, 'SS5': {'stageId': 835, 'status': 'Completed'}, 'SS6': {'stageId': 819, 'status': 'Completed'}, 'SS7': {'stageId': 833, 'status': 'Completed'}, 'SS8': {'stageId': 836, 'status': 'Cancelled'}, 'SS9': {'stageId': 820, 'status': 'Completed'}, 'SS10': {'stageId': 821, 'status': 'Completed'}, 'SS11': {'stageId': 824, 'status': 'Completed'}, 'SS12': {'stageId': 826, 'status': 'Running'}, 'SS13': {'stageId': 827, 'status': 'Running'}, 'SS14': {'stageId': 834, 'status': 'ToRun'}, 'SS15': {'stageId': 832, 'status': 'ToRun'}, 'SS16': {'stageId': 829, 'status': 'ToRun'}, 'SS17': {'stageId': 825, 'status': 'ToRun'}, 'SS18': {'stage

IntegrityError: UNIQUE constraint failed: stage_times_stage.stageTimeId

In [139]:
meta

{'rallyId': '36', 'stages': [], 'championshipId': None, 'rally_name': 'Italy'}

In [228]:
meta

NameError: name 'meta' is not defined

In [219]:
conn = sqlite3.connect(dbname)

q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [221]:
q="SELECT * FROM event_metadata LIMIT 1;"
pd.read_sql(q,conn).columns

Index(['_id', 'availability', 'date-finish', 'date-start', 'gallery',
       'hasdata' TEXT,\n  'hasvideos', 'id', 'info-based', 'info-categories',
       'info-date', 'info-flag', 'info-surface', 'info-website', 'kmlfile',
       'logo', 'name', 'org-website', 'poi-Klo im Wald', 'poilistid',
       'position', 'rosterid', 'sas-eventid', 'sas-itineraryid', 'sas-rallyid',
       'sas-trackingid', 'sitid', 'testid', 'thumbnail', 'time-zone',
       'tzoffset', 'year'],
      dtype='object')

In [ ]:
#!rm wrc18_test1.db

In [83]:
#!pip3 install isodate
import isodate
pd.to_timedelta(isodate.parse_duration('PT1H5M26S')), isodate.parse_duration('PT0.1S')
#ISO 8601 https://stackoverflow.com/questions/51168022/what-does-pt-prefix-stand-for-in-duration

(Timedelta('0 days 01:05:26'), datetime.timedelta(microseconds=100000))

In [87]:
#Data2Text - Stage Result notebook has time wrangling

AttributeError: 'Timedelta' object has no attribute 'strftime'

In [72]:
#Why does this get cast to time but stage_times_overall doesn't?
q="SELECT * FROM stage_times_stage LIMIT 10 ;"
pd.read_sql(q,conn)

#The diffFirst etc are not time objects - they;re strings; cast to timedelta? DOes SQLIte do timedelta?
#The cast is easy in Python: pd.to_timedelta('00:04:45.5000000')

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,stageTimeId,status
0,00:00:07.6000000,7600,00:00:02.3000000,2300,00:01:30.6000000,90600,3327,17,Default,849,46998,Completed
1,00:00:12.5000000,12500,00:00:01.6000000,1600,00:01:35.9000000,95900,3327,19,Default,848,46999,Completed
2,00:00:07.7000000,7700,00:00:00.1000000,100,00:01:30.7000000,90700,3326,18,Default,849,47000,Completed
3,00:00:07.8000000,7800,00:00:01,1000,00:01:31.2000000,91200,3326,17,Default,848,47001,Completed
4,00:01:50.1000000,110100,00:00:07.3000000,7300,00:14:25.7000000,865700,3327,17,Default,846,47002,Completed
5,00:00:15.6000000,15600,00:00:06.3000000,6300,00:01:38.6000000,98600,3329,20,Default,849,47003,Completed
6,00:00:50,50000,00:00:04.5000000,4500,00:08:41.1000000,521100,3327,14,Default,845,47004,Completed
7,00:08:56.1000000,536100,00:01:21.7000000,81700,00:13:39.6000000,819600,3330,25,Default,844,47005,Completed
8,00:07:05.3000000,425300,00:00:01.2000000,1200,00:08:28.3000000,508300,3331,23,Assessed,849,47006,Completed
9,00:01:01.7000000,61700,00:00:04.7000000,4700,00:05:47.2000000,347200,3329,22,Default,838,47007,Completed


In [73]:
## Are stage_times_overall and stage_overall the same?


#Need to parse these time things correctly...
#Maybe parse 'PT10M40.4S'  to  '00:10:40.4000000',  'PT1H9M30.8S' to '00:01:09:30.800000'
#0.1S ->00:00:00.100000, 1M0.1s->00:10:00.100000, 1H1M0.1S ->01:10:00.100000
q="SELECT * FROM stage_times_overall ORDER BY totalTimeMs LIMIT 10 OFFSET 200 ;"
pd.read_sql(q,conn)

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT10M40.4S,640400,PT2S,2000,3324,PT2M10S,130000,16,PT1H9M30.8S,4170800,PT1H11M40.8S,4300800,852
1,PT10M58S,658000,PT17.6S,17600,3327,PT3M,180000,17,PT1H8M58.4S,4138400,PT1H11M58.4S,4318400,852
2,PT12M14.5S,734500,PT1M16.5S,76500,3326,PT0S,0,18,PT1H13M14.9S,4394900,PT1H13M14.9S,4394900,852
3,PT23M44.8S,1424800,PT10M20.7S,620700,3330,PT2M10S,130000,21,PT1H12M22.1S,4342100,PT1H14M32.1S,4472100,846
4,PT14M4.1S,844100,PT1M49.6S,109600,3329,PT40S,40000,19,PT1H14M24.5S,4464500,PT1H15M4.5S,4504500,852
5,PT0S,0,PT0S,0,3314,PT0S,0,1,PT1H16M20.1S,4580100,PT1H16M20.1S,4580100,853
6,PT24M10.2S,1450200,PT10M37.2S,637200,3330,PT2M10S,130000,21,PT1H14M12.9S,4452900,PT1H16M22.9S,4582900,848
7,PT3.7S,3700,PT3.7S,3700,3311,PT0S,0,2,PT1H16M23.8S,4583800,PT1H16M23.8S,4583800,853
8,PT7.8S,7800,PT4.1S,4100,3310,PT0S,0,3,PT1H16M27.9S,4587900,PT1H16M27.9S,4587900,853
9,PT10S,10000,PT2.2S,2200,3312,PT0S,0,4,PT1H16M30.1S,4590100,PT1H16M30.1S,4590100,853


In [74]:
q="SELECT * FROM stage_overall ORDER BY totalTimeMs LIMIT 10;"
pd.read_sql(q,conn)

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT0S,0,PT0S,0,3313,PT0S,0,1,PT4M45.5S,285500,PT4M45.5S,285500,838
1,PT0.8S,800,PT0.8S,800,3312,PT0S,0,2,PT4M46.3S,286300,PT4M46.3S,286300,838
2,PT1.2S,1200,PT0.4S,400,3311,PT0S,0,3,PT4M46.7S,286700,PT4M46.7S,286700,838
3,PT1.8S,1800,PT0.6S,600,3308,PT0S,0,4,PT4M47.3S,287300,PT4M47.3S,287300,838
4,PT2.1S,2100,PT0.3S,300,3309,PT0S,0,5,PT4M47.6S,287600,PT4M47.6S,287600,838
5,PT2.1S,2100,PT0S,0,3314,PT0S,0,6,PT4M47.6S,287600,PT4M47.6S,287600,838
6,PT2.5S,2500,PT0.4S,400,3306,PT0S,0,7,PT4M48S,288000,PT4M48S,288000,838
7,PT2.6S,2600,PT0.1S,100,3305,PT0S,0,8,PT4M48.1S,288100,PT4M48.1S,288100,838
8,PT2.7S,2700,PT0.1S,100,3310,PT0S,0,9,PT4M48.2S,288200,PT4M48.2S,288200,838
9,PT2.8S,2800,PT0.1S,100,3307,PT0S,0,10,PT4M48.3S,288300,PT4M48.3S,288300,838


In [227]:
!ls -al *.db

-rw-r--r--  1 ajh59  1182653967    36864 12 Feb 15:32 amagicdemo.db
-rw-r--r--  1 ajh59  1182653967   638976 19 May 16:01 argentina18.db
-rw-r--r--  1 ajh59  1182653967        0 10 Feb 18:32 database.db
-rw-r--r--  1 ajh59  1182653967  1421312 31 Jul 14:08 finland18.db
-rw-r--r--  1 ajh59  1182653967   974848 19 May 12:46 france18.db
-rw-r--r--  1 ajh59  1182653967        0  8 Jun 08:30 italy.db
-rw-r--r--  1 ajh59  1182653967  1024000 11 Jun 09:46 italy18.db
-rw-r--r--  1 ajh59  1182653967   622592 19 May 12:45 mexico18.db
-rw-r--r--  1 ajh59  1182653967   966656 19 May 12:44 monaco18.db
-rw-r--r--  1 ajh59  1182653967  1290240 20 May 14:30 portugal18.db
-rw-r--r--  1 ajh59  1182653967        0  3 Feb 20:55 rally.db
-rw-r--r--  1 ajh59  1182653967  1236992 19 May 12:45 sweden18.db
-rw-r--r--  1 ajh59  1182653967  2117632  5 Mar 21:02 wrc18.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:33 wrc18_monaco.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:31 wrc18_sweden.db
-rw-

In [133]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(dbname)
c = conn.cursor()
#c.executescript(setup_q)
#c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,startlist_classes
7,penalties
8,retirements
9,stagewinners


In [134]:
q="SELECT * FROM startlists LIMIT 1;"
pd.read_sql(q,conn).to_dict()

{'codriver.abbvName': {0: 'J. INGRASSIA'},
 'codriver.code': {0: 'ING'},
 'codriver.country.countryId': {0: 76},
 'codriver.country.iso2': {0: 'FR'},
 'codriver.country.iso3': {0: 'FRA'},
 'codriver.country.name': {0: 'France'},
 'codriver.countryId': {0: 76},
 'codriver.firstName': {0: 'Julien'},
 'codriver.fullName': {0: 'Julien INGRASSIA'},
 'codriver.lastName': {0: 'INGRASSIA'},
 'codriver.personId': {0: 521},
 'codriverId': {0: 521},
 'driver.abbvName': {0: 'S. OGIER'},
 'driver.code': {0: 'OGI'},
 'driver.country.countryId': {0: 76},
 'driver.country.iso2': {0: 'FR'},
 'driver.country.iso3': {0: 'FRA'},
 'driver.country.name': {0: 'France'},
 'driver.countryId': {0: 76},
 'driver.firstName': {0: 'Sébastien\xa0'},
 'driver.fullName': {0: 'Sébastien\xa0 OGIER'},
 'driver.lastName': {0: 'OGIER'},
 'driver.personId': {0: 2453},
 'driverId': {0: 2453},
 'eligibility': {0: 'M'},
 'entrant.entrantId': {0: 94},
 'entrant.logoFilename': {0: ''},
 'entrant.name': {0: 'M-SPORT FORD WORLD RA